In [2]:

import json
import pickle
import pandas as pd
from unidecode import unidecode

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage


from langchain_core.vectorstores import InMemoryVectorStore




from langchain_openai import OpenAIEmbeddings


In [3]:
zp_typ_codes = [
    "02", # ZP pro inkontinentní pacienty"
    "02", # ZP pro inkontinentní pacienty
    "03", # ZP pro pacienty se stomií
    "04", # ZP ortopedicko-protetické a ortopedická obuv
    "05", # ZP pro pacienty s diabetem a s jinými poruchami metabolismu
    "07",  # ZP pro pacienty s poruchou mobility
    "08", #Příslušenství ke sluchadlům na zakázku
    "08", # ZP pro pacienty s poruchou sluchu,
    "09", # ZP pro korekci zraku a ZP kompenzační pro zrakově postižené na zakázku
    "09", # ZP pro pacienty s poruchou zraku
    "10", # ZP respirační, inhalační a pro aplikaci enterální výživy
    "11" # ZP nekategorizované

]

In [4]:
zp_typ_codes = list(map(lambda x: float(x), zp_typ_codes))

In [5]:
zp_typ_codes

[2.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 8.0, 9.0, 9.0, 10.0, 11.0]

In [7]:
try:
    vector_store = InMemoryVectorStore.load("data/vectorstore", embedding=OpenAIEmbeddings())
except BaseException as e:
    pass

    

In [110]:
def tokenizer(text: str) -> str:
    # coding=utf-8
    normalized = unidecode(text)
    return normalized




In [111]:
vector_store = InMemoryVectorStore(embedding=OpenAIEmbeddings())


In [117]:
# INIT vectorstorre


materialy_df = pd.read_json("data/ciselniky/materialy.jsonl", orient="recods", lines=True, encoding="utf-8")
# feed vectorstore with materials

# select only relevant records = 
# 
relevant_records = materialy_df[(materialy_df["file_code"] == "ZP") & (materialy_df["type"].isin(zp_typ_codes))]

for _, row in relevant_records.iterrows():
    
    # print(item)
    
    # implement logic to feed the vectorstore
    # feed only ZP for now 
    
    
        
    # TODO: ADD TOKENIZER TO REMOVE SPECIAL CHARACTES řěšýň...
    # Combine 'name' and 'description' attributes for page_content
    PAGE_CONTENT = f"{row.get('name', '')} {row.get('description', '')}".strip()
    
    row["record"] = "material"
    
    # Store all attributes in metadata
    doc = Document(page_content=PAGE_CONTENT, metadata=row.to_dict())
    try:
        vector_store.add_documents([doc], ids=[row["code"]])
    except BaseException as e:
        
        print(e)
        continue
        






NameError: name 'e' is not defined

TypeError: InMemoryVectorStore.dump() missing 1 required positional argument: 'path'

In [ ]:
l

In [113]:

vykony_df = pd.read_json("data/ciselniky/vykon.jsonl", orient="recods", lines=True, encoding="utf-8")

    
for _, row in vykony_df.iterrows():
            
    
    # Combine 'NAZ' and 'TXT' attributes for page_content
    PAGE_CONTENT = f"{row.get('NAZ', '')} {row.get('TEXT', '')}".strip()
    
    row["record"] = "vykon"
    # Store all attributes in metadata
    doc = Document(page_content=PAGE_CONTENT, metadata=row.to_dict())
    
    vector_store.add_documents([doc], ids=[row["KOD"]])

In [121]:
vector_store.dump("data/vectorstore")


In [8]:
def filter_funct(doc): 
    return doc.metadata.get("record") == "material"

In [156]:
from langchain_openai import OpenAI


detection_model = OpenAI()

In [ ]:
detection_model.invoke(SystemMessage(content="ve ambulantni zpravy najdi materialy, leky, vykony a zdravodnicky pomucky. use explicit text"),
        HumanMessage(content=state["text"]))

In [10]:

vector_store.similarity_search_with_score("prontosan roztok", k=20, 
    filter=filter_funct
)

[(Document(id='5001260', metadata={'code': 5001260, 'name': 'ROZTOK ELASTOVISKóZNÍ SINOVIAL ONE', 'description': 'INJ.1X2,5ML, HRAZENA 1 APLIKACE DO 1 KLOUBU/6 MĚS.', 'type': 11.0, 'limit': 'ORT, REV', 'file': 'Zdravotnické prostředky', 'file_code': 'ZP', 'record': 'material'}, page_content='ROZTOK ELASTOVISKóZNÍ SINOVIAL ONE INJ.1X2,5ML, HRAZENA 1 APLIKACE DO 1 KLOUBU/6 MĚS.'),
  0.8307802179793462),
 (Document(id='5003903', metadata={'code': 5003903, 'name': 'ROZTOK ELASTOVISKOZNÍ ORTHOVISC', 'description': 'INJ.1X2ML,ROZ.NATRIUM HYALURONÁTU 30MG/2ML,HRAZENY 3 APLIKACE DO 1 KLOUBU/6 MĚS.', 'type': 11.0, 'limit': 'ORT, REV', 'file': 'Zdravotnické prostředky', 'file_code': 'ZP', 'record': 'material'}, page_content='ROZTOK ELASTOVISKOZNÍ ORTHOVISC INJ.1X2ML,ROZ.NATRIUM HYALURONÁTU 30MG/2ML,HRAZENY 3 APLIKACE DO 1 KLOUBU/6 MĚS.'),
  0.829521145196246),
 (Document(id='5001263', metadata={'code': 5001263, 'name': 'ROZTOK ELASTOVISKóZNÍ SINOVIAL HL', 'description': 'INJ 1X2 ML, STERILNÍ ROZ